In [1]:
import pandas as pd
import mysql.connector
import streamlit as st
import logging

# Configure logging
logging.basicConfig(filename='db_upload_status.log', level=logging.INFO)

# Load the dataset
df = pd.read_csv(st.secrets["path"]["input_file"])

# Split the DataFrame into separate tables
df_orders = df[['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city', 'state', 'postal_code', 'region']]
df_products = df[['order_id', 'product_id', 'category', 'sub_category', 'cost_price', 'list_price', 'quantity', 'discount_percent', 'discount', 'sale_price', 'profit']]

# Connect to MySQL 
try:
    conn = mysql.connector.connect(
        host=st.secrets["db_config"]["server"],
        user=st.secrets["db_config"]["username"],
        password=st.secrets["db_config"]["password"],
        port=st.secrets["db_config"]["port"]
    )
    cursor = conn.cursor()

    # Check if the database exists
    cursor.execute("SHOW DATABASES LIKE %s", (st.secrets["db_config"]["database"],))
    if not cursor.fetchone():
        # Create the database if it doesn't exist
        create_db_query = f"CREATE DATABASE {st.secrets['db_config']['database']}"
        cursor.execute(create_db_query)
        logging.info(f"Database '{st.secrets['db_config']['database']}' created successfully.")

    # Use the specified database
    conn.database = st.secrets['db_config']['database']

    # Create `order_details` table (if it doesn't exist)
    create_order_table = """
    CREATE TABLE IF NOT EXISTS order_details (
        order_id INT PRIMARY KEY,
        order_date DATE,
        ship_mode VARCHAR(255),
        segment VARCHAR(255),
        country VARCHAR(255),
        city VARCHAR(255),
        state VARCHAR(255),
        postal_code VARCHAR(255),
        region VARCHAR(255)
    )
    """
    cursor.execute(create_order_table)
    logging.info("Table 'order_details' created or already exists.")

    # Create `product_details` table (if it doesn't exist)
    create_product_table = """
    CREATE TABLE IF NOT EXISTS product_details (
        order_id INT,
        product_id VARCHAR(255),
        category VARCHAR(255),
        sub_category VARCHAR(255),
        cost_price DECIMAL(10, 2),
        list_price DECIMAL(10, 2),
        quantity INT,
        discount_percent DECIMAL(5, 2),
        discount DECIMAL(10,2),
        sale_price DECIMAL(10,2),
        profit DECIMAL(10,2),
        FOREIGN KEY (order_id) REFERENCES order_details(order_id)
    )
    """
    cursor.execute(create_product_table)
    logging.info("Table 'product_details' created or already exists.")

    # Prepare data for batch insertion
    order_data = df_orders.values.tolist()
    product_data = df_products.values.tolist()

    # Insert data into `order_details` table using `executemany`
    insert_order_sql = """
    INSERT INTO order_details (order_id, order_date, ship_mode, segment, country, city, state, postal_code, region)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_order_sql, order_data)
    logging.info("Data inserted into 'order_details' table.")

    # Insert data into `product_details` table using `executemany`
    insert_product_sql = """
    INSERT INTO product_details (order_id, product_id, category, sub_category, cost_price, list_price, quantity, 
    discount_percent, discount, sale_price, profit)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_product_sql, product_data)
    logging.info("Data inserted into 'product_details' table.")

    # Commit the changes
    conn.commit()
    logging.info("Data inserted successfully!")

except mysql.connector.Error as error:
    logging.error(f"Error: {error}")
    raise  # Re-raise the exception to propagate the error

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        logging.info("MySQL connection is closed")